In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

# Загрузим объединённый датафреймы

valid_df = pd.read_parquet('../vkrecsys/df_valid.parquet')
cat_valid = pd.read_csv('../vkrecsys/cat_valid.csv')
als_valid = pd.read_csv('../vkrecsys/als_valid.csv')

In [15]:
new_df = pd.merge(valid_df, cat_valid, on=["user_id", "item_id"])

In [3]:
new_df = pd.merge(new_df, als_valid, on=["user_id", "item_id"])

In [16]:
new_df.head()

,user_id,item_id,gender,age,predict
0,3681,96209,2,44,0.966992
1,10210,45267,2,34,0.972160
2,164071,44494,1,22,0.970090
3,16889,88557,2,38,0.987155
4,57099,139241,2,43,0.988969


In [17]:
valid_stack = pd.read_parquet('../vkrecsys/df_valid_stack.parquet')

In [18]:
valid_stack.head()

,user_id,item_id,like
0,3681,96209,1
1,10210,45267,1
2,164071,44494,1
3,16889,88557,1
4,57099,139241,1


In [19]:
new_df = pd.merge(new_df, valid_stack, on=["user_id", "item_id"])

In [20]:
new_df.head()

,user_id,item_id,gender,age,predict,like
0,3681,96209,2,44,0.966992,1
1,10210,45267,2,34,0.972160,1
2,164071,44494,1,22,0.970090,1
3,16889,88557,2,38,0.987155,1
4,57099,139241,2,43,0.988969,1


In [21]:
new_df.drop(columns=['gender', 'age'], inplace=True)
new_df.head()

,user_id,item_id,predict,like
0,3681,96209,0.966992,1
1,10210,45267,0.972160,1
2,164071,44494,0.970090,1
3,16889,88557,0.987155,1
4,57099,139241,0.988969,1


In [22]:
new_df.tail(3)

,user_id,item_id,predict,like
3196952,30695,25495,0.706199,0
3196953,408,179282,0.716101,1
3196954,72876,168799,0.773378,0


In [23]:
# Загрузка данных из files
#users_meta = pd.read_parquet("../vkrecsys/users_meta.parquet.parquet")
test_pairs = pd.read_csv("../vkrecsys/test_pairs.csv.csv")
cat_real = pd.read_csv('../vkrecsys/cat_real.csv')
als_real = pd.read_csv('../vkrecsys/als_real.csv')
# Объединяем таблицы по полю user_id
#joined_table = test_pairs.merge(users_meta, on="user_id", how="left")

In [24]:
test_pairs = pd.merge(test_pairs, cat_real, on=["user_id", "item_id"])
#test_pairs = pd.merge(test_pairs, als_real, on=["user_id", "item_id"])

In [25]:
test_pairs.head()

,user_id,item_id,predict
0,1,7363,0.742713
1,1,73770,0.873571
2,1,75700,0.739480
3,1,81204,0.657423
4,1,110249,0.539389


In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
# Загрузка данных
#data = pd.read_csv('validation_dataset.csv')
data = new_df
# Разделение на признаки и цели
X = data[['predict']]
y = data['like']

# Разделение на тренировочную и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Обучение метамодели
meta_model = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)
meta_model.fit(X_train, y_train)

# Загрузка новых данных
new_data = test_pairs   # Предполагается, что new_data содержит predict_x и predict_y
# Извлечение признаков из новых данных
new_X = new_data[['predict']]
# Предсказание на новых данных
new_predictions = meta_model.predict(new_X)
# Сохранение предсказаний в отдельный DataFrame
new_data['predict'] = new_predictions
# Вывод результатов
print(new_data[['user_id', 'item_id', 'predict']])
# Расчет вероятностей на тестовой выборке
probabilities = meta_model.predict_proba(X_test)[:, 1]
# Оценка качества модели по ROC-AUC
roc_auc = roc_auc_score(y_test, probabilities)
print(f'ROC-AUC: {roc_auc:.4f}')

KeyError: "None of [Index(['predict_x', 'predict_y'], dtype='object')] are in the [columns]"

In [28]:
# Загрузка новых данных
new_data = test_pairs   # Предполагается, что new_data содержит predict_x и predict_y
# Извлечение признаков из новых данных
new_X = new_data[['predict']]
# Предсказание на новых данных
new_predictions = meta_model.predict(new_X)
# Сохранение предсказаний в отдельный DataFrame
new_data['predict'] = new_predictions
# Вывод результатов
print(new_data[['user_id', 'item_id', 'predict']])
# Расчет вероятностей на тестовой выборке
probabilities = meta_model.predict_proba(X_test)[:, 1]
# Оценка качества модели по ROC-AUC
roc_auc = roc_auc_score(y_test, probabilities)
print(f'ROC-AUC: {roc_auc:.4f}')

         user_id  item_id  predict
0              1     7363        1
1              1    73770        1
2              1    75700        1
3              1    81204        1
4              1   110249        1
...          ...      ...      ...
1655115   183403   235097        1
1655116   183403   267411        0
1655117   183403   273646        0
1655118   183403   288305        0
1655119   183403   322029        0

[1655120 rows x 3 columns]
ROC-AUC: 0.7832


In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

In [15]:
# Загрузка данных
#data = pd.read_csv('validation_dataset.csv')
data = new_df
# Разделение на признаки и цели
X = data[['predict_x', 'predict_y']]
y = data['like']

# Разделение на тренировочную и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
# Создание и обучение метамодели (логистической регрессии)
meta_model = LogisticRegression()
meta_model.fit(X_train, y_train)

LogisticRegression()

In [17]:
# Прогнозирование на тестовой выборке
predictions = meta_model.predict(X_test)
probabilities = meta_model.predict_proba(X_test)[:, 1]

# Оценка точности
accuracy = accuracy_score(y_test, predictions)
roc_auc = roc_auc_score(y_test, probabilities)

print(f'Accuracy: {accuracy:.4f}')
print(f'ROC-AUC: {roc_auc:.4f}')

Accuracy: 0.8256
ROC-AUC: 0.7772


In [29]:
# Загрузка новых данных
new_data = test_pairs   # Предполагается, что new_data содержит predict_x и predict_y

# Извлечение признаков из новых данных
new_X = new_data[['predict_x', 'predict_y']]

# Предсказание на новых данных
new_predictions = meta_model.predict(new_X)

# Сохранение предсказаний в отдельный DataFrame
new_data['predict'] = new_predictions

# Вывод результатов
print(new_data[['user_id', 'item_id', 'predict']])

         user_id  item_id  predict
0              1     7363        1
1              1    73770        1
2              1    75700        1
3              1    81204        1
4              1   110249        1
...          ...      ...      ...
1655115   183403   235097        1
1655116   183403   267411        0
1655117   183403   273646        0
1655118   183403   288305        0
1655119   183403   322029        0

[1655120 rows x 3 columns]


In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Загрузка данных
data = new_df

# Разделение на признаки и цели
X = data[['predict_x', 'predict_y']]
y = data['like']

# Разделение на тренировочную и тестовую выборку
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Обучение метамодели
meta_model = LogisticRegression()
meta_model.fit(X_train, y_train)

# Расчет вероятностей на тестовой выборке
probabilities = meta_model.predict_proba(X_test)[:, 1]

# Оценка качества модели по ROC-AUC
roc_auc = roc_auc_score(y_test, probabilities)
print(f'ROC-AUC: {roc_auc:.4f}')

# Загрузка новых данных
new_data = test_pairs  # Предполагается, что new_data содержит predict_x и predict_y

# Извлечение признаков из новых данных
new_X = new_data[['predict_x', 'predict_y']]

# Предсказание вероятностей на новых данных
new_probabilities = meta_model.predict_proba(new_X)[:, 1]

# Сохранение вероятностей в отдельный DataFrame
new_data['predict'] = new_probabilities

# Вывод результатов
print(new_data[['user_id', 'item_id', 'predict']])

ROC-AUC: 0.7772
         user_id  item_id   predict
0              1     7363  0.863763
1              1    73770  0.938382
2              1    75700  0.861195
3              1    81204  0.781725
4              1   110249  0.619004
...          ...      ...       ...
1655115   183403   235097  0.831924
1655116   183403   267411  0.498126
1655117   183403   273646  0.388875
1655118   183403   288305  0.246422
1655119   183403   322029  0.348999

[1655120 rows x 3 columns]


In [31]:
new_data.to_csv("../vkrecsys/fin_8.csv", index=False)